Fama-French model:시장(MKT), 크기(SMB), 가치(HML), 수익성(RMW), 투자(5-factor의 CMA) 등 여러 요인을 기반으로 주식 포트폴리오의 수익률을 설명 
Fama-French 5-factor 모델을 적용하여, 한국 상장기업의 주식 수익률을 시장, 규모, 가치, 수익성, 투자 요인으로 분석하고 파이썬으로 구현하고자 함
->우선적으로 식품, 자동차 섹터만 돌려보았음

터미널 설치

In [1]:
pip install pykrx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


A. SMG(Small-Big): 

In [3]:
#1.상위50%,하위50% 기업 리스트 정리
import pandas as pd

# CSV 파일 정확한 경로 지정
file_path = r'C:\Users\김주원\Desktop\식품, 자동차 섹터.csv'

# CSV 파일 불러오기
df = pd.read_csv(file_path, encoding='cp949')

# 시가총액 전처리: ₩, 쉼표, \ 제거 후 숫자형으로 변환
df['시가총액'] = df['시가총액'].astype(str)
df['시가총액'] = df['시가총액'].str.replace('₩', '', regex=False)
df['시가총액'] = df['시가총액'].str.replace(',', '', regex=False)
df['시가총액'] = df['시가총액'].str.replace('\\', '', regex=False)

# 문자열 → 숫자형 (오류 시 NaN 처리)
df['시가총액'] = pd.to_numeric(df['시가총액'], errors='coerce')

# 중앙값 계산 후 NaN 제외하고 상위/하위 50% 분류
median_value = df['시가총액'].median()
df['규모구분'] = df['시가총액'].apply(
    lambda x: '상위50%' if pd.notnull(x) and x >= median_value
    else ('하위50%' if pd.notnull(x) else None)
)

# 결과 미리보기
print(df[['종목명', '시가총액', '규모구분']].head())

# 결과 저장
output_path = r'C:\Users\김주원\Desktop\test_분류결과.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')


         종목명            시가총액   규모구분
0        케이카    749713055880  상위50%
1     KG모빌리티    735632805120  상위50%
2      KG케미칼    336867676800  상위50%
3         기아  39608194147200  상위50%
4  코오롱모빌리티그룹    161023646250  상위50%


In [4]:
#2. 월말 종가데이터(네이버금융으로 크롤링하려고 하였으나 종가,시가총액 데이터가 잘 뽑히지 않는 이슈로 pykrx로 뽑음)
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 1. 파일 읽기 (종목코드 str로 읽기)
file_path = r"C:\Users\김주원\Desktop\식품, 자동차 섹터.csv"
df = pd.read_csv(file_path, dtype={'종목코드': str}, encoding='cp949')
df['종목코드'] = df['종목코드'].str.zfill(6)  # 항상 6자리로 맞추기

codes = df['종목코드'].tolist()

# 월말 종가 크롤링 함수
def get_monthly_data(stock_code, year, month):
    url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    result = []
    for page in range(1, 4):
        pg_url = f"{url}&page={page}"
        res = requests.get(pg_url, headers={"User-Agent":"Mozilla/5.0"})
        soup = BeautifulSoup(res.text, "html.parser")
        table = soup.select("table[type='N']")
        if not table:
            continue
        rows = table[0].select("tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 7:
                continue
            date = cols[0].text.strip().replace('.', '-')
            if not date:
                continue
            try:
                y, m, d = map(int, date.split('-'))
                if y == year and m == month:
                    close = int(cols[1].text.replace(',', ''))
                    result.append((date, close))
            except:
                continue
    if not result:
        return None, None
    result.sort()
    return result[-1]

# 시가총액, PBR 크롤링 함수
def get_valuation(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    res = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(res.text, "html.parser")
    try:
        market_cap_tag = soup.select_one('em#_market_sum')
        market_cap = market_cap_tag.text.strip().replace(",", "") if market_cap_tag else None
       
        pbr = None
        for th in soup.select('table.per_table th'):
            if 'PBR' in th.text:
                em = th.find_next('td').find('em')
                pbr = em.text.strip() if em else None
                break
    except Exception as e:
        market_cap, pbr = None, None
    return market_cap, pbr

# 크롤링 실행
result_list = []
for code in tqdm(codes):
    y, m = 2024, 6  # 원하는 연도와 월
    date, close = get_monthly_data(code, y, m)
    market_cap, pbr = get_valuation(code)
    name = df[df['종목코드'] == code]['종목명'].values[0]
    result_list.append({
        '종목코드': code,
        '종목명': name,
        '월말일자': date,
        '월말종가': close,
        '시가총액': market_cap,
        'PBR': pbr
    })

result_df = pd.DataFrame(result_list)
result_df.to_csv(r"C:\Users\김주원\Desktop\네이버월말_식품,자동차_데이터.csv", index=False, encoding='utf-8-sig')
print("완료!")

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [00:58<00:00,  1.82it/s]

완료!


In [ ]:
# 2 월말 종가/시총 데이터(pykrx버전)
from pykrx import stock
import pandas as pd
from tqdm import tqdm

file_path = r"C:\Users\김주원\Desktop\식품, 자동차 섹터.csv"
df = pd.read_csv(file_path, dtype={'종목코드': str}, encoding='cp949')
df['종목코드'] = df['종목코드'].str.zfill(6)
codes = df['종목코드'].tolist()

result = []
for code in tqdm(codes):
    name = df[df['종목코드'] == code]['종목명'].values[0]
    for year in range(2000, 2025):
        for month in range(1, 13):
            date = pd.to_datetime(f"{year}-{month:02d}-01") + pd.offsets.MonthEnd(0)
            try:
                close = stock.get_market_ohlcv_by_date(date, date, code)['종가'].values[0]
                mcap = stock.get_market_cap_by_date(date, date, code)['시가총액'].values[0]
                result.append({
                    '종목코드': code,
                    '종목명': name,
                    '연도': year,
                    '월': month,
                    '월말일자': date.strftime('%Y-%m-%d'),
                    '월말종가': close,
                    '월말시가총액': mcap
                })
            except:
                continue

result_df = pd.DataFrame(result)
result_df.to_csv(r"C:\Users\김주원\Desktop\월말_식품자동차_데이터_pykrx.csv", index=False, encoding='utf-8-sig')
print("완료!")


 62%|██████████████████████████████████████████████████▍                              | 66/106 [48:47<28:04, 42.10s/it]

B.HML(High-Low)가치: 주식의 장부가치(Book value = 순자산 = 자본)을 시가총액으로 나눈 값 = 1/PBR

In [ ]:
# pbr 구하기 
from pykrx import stock
import pandas as pd
from tqdm import tqdm

# 종목코드 리스트 읽기 (예시: CSV 파일)
file_path = r"C:\Users\김주원\Desktop\식품, 자동차 섹터.csv"
df = pd.read_csv(file_path, dtype={'종목코드': str}, encoding='cp949')
df['종목코드'] = df['종목코드'].str.zfill(6)
codes = df['종목코드'].tolist()

result = []
for code in tqdm(codes):
    name = df[df['종목코드'] == code]['종목명'].values[0]
    for year in range(2000, 2024): 
        for month in range(1, 13):
            date = pd.to_datetime(f"{year}-{month:02d}-01") + pd.offsets.MonthEnd(0)
            try:
                pbr = stock.get_market_fundamental_by_date(date, date, code)['PBR'].values[0]
                if pbr == 0 or pd.isna(pbr):
                    bm = None
                else:
                    bm = 1 / pbr
                result.append({
                    '종목코드': code,
                    '종목명': name,
                    '연도': year,
                    '월': month,
                    '월말일자': date.strftime('%Y-%m-%d'),
                    'PBR': pbr,
                    'B/M(1/PBR)': bm
                })
            except Exception as e:
                continue

result_df = pd.DataFrame(result)
result_df.to_csv(r"C:\Users\김주원\Desktop\월말_PBR_BM_2000_2025.csv", index=False, encoding='utf-8-sig')
print("완료!")


C.RMW(Robust-Weak)

In [ ]:
from OpenDartReader import OpenDartReader
import pandas as pd
import time

# 1. 종목코드 리스트 CSV에서 불러오기
file_path = r"C:\Users\김주원\Desktop\식품, 자동차 섹터.csv"
df_code = pd.read_csv(file_path, dtype={'종목코드': str}, encoding='cp949')
df_code['종목코드'] = df_code['종목코드'].str.zfill(6)
code_list = df_code['종목코드'].tolist()

# 2. DART API 키 입력
api_key = "8a0be11993744250453be56e4cfeb903eec9e608"
dart = OpenDartReader(api_key)

# 3. 계정명 후보 리스트(재무제표에 계정명이 동일하게 적혀져있지 않는 경우 존재하기 때문에 가능한 후보 리스트 추가하여 인식안되는 경우 방지)
매출액_가능명 = ['매출액', '수익', '영업수익', '매출', '영업매출액']
매출원가_가능명 = ['매출원가', '영업원가']
판관비_가능명 = ['판매비와관리비', '판관비', '판매관리비']
이자비용_가능명 = ['이자비용']
자본총계_가능명 = ['자본총계', '자본']

def get_value(df, name_list):
    for name in name_list:
        v = df.loc[df['account_nm'].str.contains(name), 'thstrm_amount']
        if not v.empty:
            try:
                return float(v.values[0].replace(',', ''))
            except:
                return None
    return None

result = []

for code in code_list:
    corp_code = dart._find_corp_code(code)
    for year in range(2000, 2025):
        try:
            fs = dart.finstate(corp=corp_code, bsns_year=str(year), reprt_code='11011')
            if fs is None or fs.empty:
                continue
            sales = get_value(fs, 매출액_가능명)
            cogs = get_value(fs, 매출원가_가능명)
            sgna = get_value(fs, 판관비_가능명)
            interest = get_value(fs, 이자비용_가능명)
            equity = get_value(fs, 자본총계_가능명)
            if None in [sales, cogs, sgna, interest, equity]:
                op = None
            else:
                op = (sales - cogs - sgna - interest) / equity
            result.append({
                '종목코드': code,
                '연도': year,
                '매출액': sales,
                '매출원가': cogs,
                '판관비': sgna,
                '이자비용': interest,
                '자본총계': equity,
                'OP': op
            })
            time.sleep(0.2)
        except Exception as e:
            print(f"{code} {year} 오류: {e}")
            continue

# 결과 저장
result_df = pd.DataFrame(result)
result_df.to_csv("연도별_OP_2000_2024.csv", index=False, encoding='utf-8-sig')
print("완료!")


D.CMA(Conservative-Aggressive)

E.Rm-Rf:전체 시장의 평균 수익률에서 무위험 수익률(국채 수익률)을 뺀 시장 초과 수익률

*참고: https://psystat.tistory.com/147,
https://velog.io/@hyksun2015/%ED%8C%8C%EB%A7%88-%ED%94%84%EB%A0%8C%EC%B9%98-5%ED%8C%A9%ED%84%B0,
https://github.com/hyunyulhenry/quant_py/blob/main/data_korea.ipynb
